## Exploring the Relationship Between Airbnb Listings and Interest Points in NYC
#### Author: Tommy Polanco, Email: tp2523@columbia.edu

Is there a traveler seeking lodging who does not know of Airbnb today? Since the company's founding in 2008, Airbnb has become an industry-altering platform primarily for those seeking to arrange or offer lodging services. In New York City alone, there are around forty to fifty thousand Airbnb home listings, and this figure will only be expected to increase as the service continues to rise in popularity and usage. 

With the mark of Airbnb on New York City becoming clearer each day, I began to wonder about the price determinants of Airbnb home listings -- in particular, I was interested in the effects of neighboring interest points. (A [research paper](https://www.researchgate.net/publication/319865700_Key_Factors_Affecting_the_Price_of_Airbnb_Listings_A_Geographically_Weighted_Approach) listed it as a key factor in determining price) Given the availability of open source data for Airbnb homes and listed landmarks/interest points (Examples include parks, transportation centers, museums, avenues, churches, and so forth) in NYC, exploring this thought through data became a feasible option.

**The two main sources of data I used (exported as excel files and uploaded to a SQL database) for my project are the following:**
- [New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data) from Kaggle.
- [Points of Interest](https://data.cityofnewyork.us/City-Government/Points-Of-Interest/rxuy-2muj) from NYC Open Data.  

**This project is divided into three parts in which I use, in respective order, SQL, Python, and Tableau.**
- **SQL**: To check and clean my datasets, update columns and their values, and perform necessary joins. **This was done first. If you're interested in reading the .sql file, please [click here](https://github.com/TPolanco/Exploring-Airbnb-Interest-Points-NYC/blob/master/airbnb_interestpoints.sql). For the query output file, please [click here](https://github.com/TPolanco/Exploring-Airbnb-Interest-Points-NYC/blob/master/query_output.rpt).**
- **Python**: To perform further data manipulation and implement a necessary data structure to perform a range search. **Continue below to read the code.** 
- **Tableau**: To visualize the finalized data. **You can check it out [here](https://public.tableau.com/en-us/s/).**

If you have any questions or would like to reach out in general, please feel free to do so! My email is on top of this notebook.

In [1]:
##Let's begin
#we will use these libraries for any necessary data manipulation
import numpy as np
import pandas as pd

#contains connection to the SQL database from where our datasets are
import database

In [2]:
#get our airbnb table into a dataframe
airbnb = pd.read_sql('SELECT * FROM dbo.airbnb', database.conn)

#display the first 7 rows of the dataframe
airbnb.head(7)

,id,name,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,review_count,last_review,reviews_per_month,host_listings_count,availability_365
0,31447607.0,SUN FILLED APARTMENT WITH STUNNING RIVER VIEWS,Manhattan,Hell's Kitchen,40.76666,-73.98358,Entire home/apt,225.0,30.0,0.0,NaT,0.00,12.0,343.0
1,31447999.0,Stunning Modern 1br with Private Balcony & Views,Manhattan,Midtown,40.76616,-73.98228,Entire home/apt,225.0,30.0,0.0,NaT,0.00,12.0,365.0
2,31448783.0,Sleek 1 Bedroom Brownstone Apartment in Brooklyn,Brooklyn,Bedford-Stuyvesant,40.67890,-73.91014,Entire home/apt,80.0,2.0,8.0,2019-07-07,2.12,1.0,0.0
3,31450131.0,Apt2- large bedroom.,Brooklyn,Brownsville,40.66262,-73.91575,Private room,59.0,1.0,2.0,2019-05-31,0.42,2.0,53.0
4,31453103.0,Newly Renovated Cozy Private 1 Bedroom Apartment,Brooklyn,East Flatbush,40.64656,-73.95016,Entire home/apt,65.0,2.0,20.0,2019-06-29,3.57,2.0,293.0
5,31457187.0,Brooklyn Bliss V,Brooklyn,East Flatbush,40.65808,-73.93903,Private room,39.0,2.0,3.0,2019-05-08,0.73,2.0,67.0
6,31458142.0,Huge studio in the heart of Kips Bay,Manhattan,Kips Bay,40.74343,-73.98147,Entire home/apt,140.0,1.0,3.0,2019-04-01,0.63,1.0,0.0


In [3]:
#do the same for interest points table
interests = pd.read_sql('SELECT * FROM dbo.interest_points', database.conn)

interests.head(7)

,the_geom,BOROUGH,FACILITY_T,SOURCE,NAME
0,POINT (-73.81678346994171 40.78815244048363),Queens,Transportation,DCP,14 AV OV BELT CROSS ISL PKWY
1,POINT (-73.82615714925024 40.75979346986441),Queens,Transportation,DCP,UNION ST OVER LIRR PORT WASH BR
2,POINT (-73.81611636010126 40.76254832987148),Queens,Transportation,DCP,149 ST BR OVR LIRR PORT WASH BR
3,POINT (-73.90009437988503 40.743127348304334),Queens,Transportation,DCP,65 STREET OVER LIRR
4,POINT (-73.89208418942773 40.742917689833284),Queens,Transportation,DCP,WOODSIDE AVE OVER CSX TRANSPORT
5,POINT (-73.84692456827754 40.67036293740125),Queens,Transportation,DCP,WHITELAW ST PEDESTRIAN
6,POINT (-73.93118460874322 40.714284752256745),Brooklyn,Transportation,OTHER,METROPOLITAN AVENUE


In order to do any appropriate analysis requiring the coordinates between Airbnb locations and NYC interest points, we need to split the "the_geom" column into two separate columns: "latitude" and "longitude". Fortunately, pandas makes this easy for us to do.

In [4]:
#we can split 'the_geom' at three boundaries: at 'POINT (', at the whitespace ' ', and at ')'.
#from these three resulting columns, longitude will be at the second and latitude will be at the third
interests['longitude'] = interests['the_geom'].str.split('POINT \(| |\)', expand=True)[1]
interests['latitude'] = interests['the_geom'].str.split('POINT \(| |\)', expand=True)[2]

#drop 'the_geom' column
interests.drop(['the_geom'],axis=1, inplace=True)

In [5]:
#let's observe the changes
interests.head(7)

,BOROUGH,FACILITY_T,SOURCE,NAME,longitude,latitude
0,Queens,Transportation,DCP,14 AV OV BELT CROSS ISL PKWY,-73.81678346994171,40.78815244048363
1,Queens,Transportation,DCP,UNION ST OVER LIRR PORT WASH BR,-73.82615714925024,40.75979346986441
2,Queens,Transportation,DCP,149 ST BR OVR LIRR PORT WASH BR,-73.81611636010126,40.76254832987148
3,Queens,Transportation,DCP,65 STREET OVER LIRR,-73.90009437988503,40.743127348304334
4,Queens,Transportation,DCP,WOODSIDE AVE OVER CSX TRANSPORT,-73.89208418942773,40.742917689833284
5,Queens,Transportation,DCP,WHITELAW ST PEDESTRIAN,-73.84692456827754,40.67036293740125
6,Brooklyn,Transportation,OTHER,METROPOLITAN AVENUE,-73.93118460874322,40.714284752256745


In [6]:
#Find out total rows in both tables
print("Number of rows in airbnb table:", airbnb.shape[0])
print("Number of rows in interest points table:", interests.shape[0])

Number of rows in airbnb table: 48895
Number of rows in interest points table: 19861


From the interests dataframe, let's create two more tables that are grouped solely by commercial and transportation points of interest. In our later analysis we will observe these two isolated variables.

In [7]:
#create tables for commercial and transportation only interest points
interests_commercial = interests[interests['FACILITY_T'].str.match('Commercial')]
interests_transport = interests[interests['FACILITY_T'].str.match('Transportation')]

In [8]:
#Let's observe the table
interests_commercial.head(7)

,BOROUGH,FACILITY_T,SOURCE,NAME,longitude,latitude
999,Staten Island,Commercial,EMS,TARGET STORES,-74.23256556441474,40.52846866385151
1015,Manhattan,Commercial,NYPD,LUCERNE HOTEL,-73.97840452035432,40.783487673936826
1039,Brooklyn,Commercial,DoITT,BROOKLYN NAVY YARD BUILDING 58,-73.97367546775524,40.69865168581588
1064,Brooklyn,Commercial,OTHER,SEA TRAVELERS BOAT SALES,-73.91362606675423,40.60227925020982
1070,Brooklyn,Commercial,DCP,BK WHOLESALE MEAT MARKET BLDG C,-74.02426388270966,40.64953283683197
1204,Brooklyn,Commercial,EMS,GREGORY HOTEL,-74.02741087003307,40.624405638515285
1238,Manhattan,Commercial,NYPD,PLAZA HOTEL CONDOMINIUM,-73.97412861492437,40.76433213874111


In [9]:
#Let's observe the table
interests_transport.head(7)

,BOROUGH,FACILITY_T,SOURCE,NAME,longitude,latitude
0,Queens,Transportation,DCP,14 AV OV BELT CROSS ISL PKWY,-73.81678346994171,40.78815244048363
1,Queens,Transportation,DCP,UNION ST OVER LIRR PORT WASH BR,-73.82615714925024,40.75979346986441
2,Queens,Transportation,DCP,149 ST BR OVR LIRR PORT WASH BR,-73.81611636010126,40.76254832987148
3,Queens,Transportation,DCP,65 STREET OVER LIRR,-73.90009437988503,40.743127348304334
4,Queens,Transportation,DCP,WOODSIDE AVE OVER CSX TRANSPORT,-73.89208418942773,40.742917689833284
5,Queens,Transportation,DCP,WHITELAW ST PEDESTRIAN,-73.84692456827754,40.67036293740125
6,Brooklyn,Transportation,OTHER,METROPOLITAN AVENUE,-73.93118460874322,40.714284752256745


Let's also generate a dataframe with descriptive statistics for prices of Airbnbs throughout neighborhoods in NYC. 

In [10]:
#table of descriptive statistics for prices of airbnbs throughout NYC
airbnb_grouping = airbnb['price'].groupby([airbnb['borough'], airbnb['neighbourhood']])
airbnb_stats = airbnb_grouping.describe().reset_index()

airbnb_stats

,borough,neighbourhood,count,mean,std,min,25%,50%,75%,max
0,Bronx,Allerton,42.0,87.595238,73.048617,33.0,47.00,66.5,104.75,450.0
1,Bronx,Baychester,7.0,75.428571,17.415100,53.0,64.50,75.0,85.00,101.0
2,Bronx,Belmont,24.0,77.125000,68.949454,24.0,30.00,52.5,99.25,299.0
3,Bronx,Bronxdale,19.0,57.105263,20.283805,25.0,41.50,50.0,75.50,100.0
4,Bronx,Castle Hill,9.0,63.000000,33.462666,38.0,39.00,39.0,86.00,128.0
5,Bronx,City Island,18.0,173.000000,240.211916,29.0,84.00,97.5,117.50,1000.0
6,Bronx,Claremont Village,28.0,87.464286,70.065833,26.0,48.25,68.5,113.00,390.0
7,Bronx,Clason Point,21.0,112.761905,101.554372,40.0,46.00,80.0,110.00,379.0
8,Bronx,Co-op City,2.0,77.500000,3.535534,75.0,76.25,77.5,78.75,80.0
9,Bronx,Concourse,50.0,86.180000,48.869543,35.0,46.00,81.0,104.25,250.0


## Searching for Neighboring Points of Airbnb Locations

Let's say I've found an Airbnb in NYC that I am looking to rent. I happen to be interested in how many points of interest are located within some circular radius from my specific Airbnb -- how could I solve this without manually pinpointing each point in a map? This is essentially a [range search problem](https://en.wikipedia.org/wiki/Range_searching), and one efficient approach we can take -- given that we have knowledge concerning the locations of all interest points -- is to feed our data into a [k-dimensional tree](https://en.wikipedia.org/wiki/K-d_tree) and use it to run a search algorithm for all neighboring points within a given radius. The k-d tree is a useful type of binary search tree that allows us to make quick searches -- with an average time complexity of $\mathcal{O}(\log n)$ -- when dealing with coordinate points.

Before we can implement our k-d tree to run searches through our coordinate data (which is in decimal degrees), we need to determine the distances we want to use for our radius and convert them to the appropriate units. 

Assuming the average person walks at a pace of 1.4 meters per second (m/s) on a crosswalk and sidewalk -- we can use the following times to calculate various distances in meters for our circle's radius:
- 1 minute walk = 84 meters
$$\left(\frac{1.4 m}{1.0 s}\right) \left(\frac{60 s}{1 min}\right) \left(1 min \right) = 84 m$$
- 5 minute walk = 420 meters
$$\left(\frac{1.4 m}{1.0 s}\right) \left(\frac{60 s}{1 min}\right) \left(5 min \right) = 420 m$$
- 10 minute walk = 840 meters
$$\left(\frac{1.4 m}{1.0 s}\right) \left(\frac{60 s}{1 min}\right) \left(10 min \right) = 840 m$$

Now our final step is to find a suitable conversion from meters to decimal degrees in order to conduct neighbor searches with our data points in the k-d tree. Detailed information, as well as a table of conversions (keep in mind that the conversions are approximations) can be found [here](https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude). 

The corresponding conversions for our three distances in decimal degrees (up to six decimal places) are:
- 1 minute walk = 84 meters = 0.000757 degrees
- 5 minute walk = 420 meters = 0.003818 degrees
- 10 minute walk = 840 meters = 0.007636 degrees

For each Airbnb location, let's now implement a range search on:
- All interest points.
- Commercial interest points.
- Transportation interest points.

### All Interest Points
#### 1 Minute Walk Case

In [11]:
#import packages needed for k-d tree
from scipy.spatial import cKDTree
from scipy import inf

#create a column for airbnb table
#1. total number of interest points within 1 minute walk
airbnb["interests_1min_walk"] = 0

#lists of all airbnb coordinates and interest points coordinates
airbnb_coordinates = list(zip(airbnb.latitude, airbnb.longitude))
interest_coordinates = list(zip(interests.latitude, interests.longitude))

#need to join the airbnb coordinates with the interest points coordinates
#to obtain a single list of combined coordinates that can be placed in the k-d tree
#for faster querying
combined_coordinates = interest_coordinates + airbnb_coordinates
tree = cKDTree(combined_coordinates)

#converting latitude and longitude columns in airbnb to sets
#sets are fast for value searches and will come in handy in our for loop
airbnb_latitude_set = set(airbnb['latitude'])
airbnb_longitude_set = set(airbnb['longitude'])

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.000757)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.000757)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #augment counter below only if it's an interest point
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new column with total count of corresponding neighbors
    airbnb.at[counter, "interests_1min_walk"] = nearby_int_pts 

#### 5 Minute Walk Case

In [12]:
#create one column for airbnb table
#1. total number of interest points within 5 minute walk
airbnb["interests_5min_walk"] = 0

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.003818)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.003818)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #augment counter below only if it's an interest point
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new column with total count of corresponding neighbors
    airbnb.at[counter, "interests_5min_walk"] = nearby_int_pts

#### 10 Minute Walk Case

In [13]:
#create one column for airbnb table
#1. total number of interest points within 10 minute walk
airbnb["interests_10min_walk"] = 0

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.007636)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.007636)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #augment counter below if it's an interest point
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new column with total count of corresponding neighbors
    airbnb.at[counter, "interests_10min_walk"] = nearby_int_pts 

In [14]:
#let's observe the changes
airbnb.head(7)

,id,name,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,review_count,last_review,reviews_per_month,host_listings_count,availability_365,interests_1min_walk,interests_5min_walk,interests_10min_walk
0,31447607.0,SUN FILLED APARTMENT WITH STUNNING RIVER VIEWS,Manhattan,Hell's Kitchen,40.76666,-73.98358,Entire home/apt,225.0,30.0,0.0,NaT,0.00,12.0,343.0,1,38,213
1,31447999.0,Stunning Modern 1br with Private Balcony & Views,Manhattan,Midtown,40.76616,-73.98228,Entire home/apt,225.0,30.0,0.0,NaT,0.00,12.0,365.0,0,40,186
2,31448783.0,Sleek 1 Bedroom Brownstone Apartment in Brooklyn,Brooklyn,Bedford-Stuyvesant,40.67890,-73.91014,Entire home/apt,80.0,2.0,8.0,2019-07-07,2.12,1.0,0.0,1,18,75
3,31450131.0,Apt2- large bedroom.,Brooklyn,Brownsville,40.66262,-73.91575,Private room,59.0,1.0,2.0,2019-05-31,0.42,2.0,53.0,0,19,100
4,31453103.0,Newly Renovated Cozy Private 1 Bedroom Apartment,Brooklyn,East Flatbush,40.64656,-73.95016,Entire home/apt,65.0,2.0,20.0,2019-06-29,3.57,2.0,293.0,0,9,40
5,31457187.0,Brooklyn Bliss V,Brooklyn,East Flatbush,40.65808,-73.93903,Private room,39.0,2.0,3.0,2019-05-08,0.73,2.0,67.0,0,20,61
6,31458142.0,Huge studio in the heart of Kips Bay,Manhattan,Kips Bay,40.74343,-73.98147,Entire home/apt,140.0,1.0,3.0,2019-04-01,0.63,1.0,0.0,3,42,172


### Commercial Interest Points
#### 1 Minute Walk Case

In [15]:
#create one column for airbnb table
#total number of commercial interest points within 1 minute walk
airbnb["commercial_pts_1min_walk"] = 0

#list of all commercial interest points coordinates
#previous airbnb_coordinates remains unchanged
interest_coordinates = list(zip(interests_commercial.latitude, interests_commercial.longitude))

#need to join the airbnb coordinates with the interest points coordinates
#to obtain a single list of combined coordinates that can be placed in the k-d tree
#for faster querying
combined_coordinates = interest_coordinates + airbnb_coordinates
tree = cKDTree(combined_coordinates)

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.000757)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.000757)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #and augment either of the two counters by each one found
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new columns with total count of corresponding neighbors
    airbnb.at[counter, "commercial_pts_1min_walk"] = nearby_int_pts

#### 5 Minute Walk Case

In [16]:
#create one column for airbnb table
#1. total number of commercial points within 5 minute walk
airbnb["commercial_pts_5min_walk"] = 0

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.003818)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.003818)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #augment counter below only if it's an interest point
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new column with total count of corresponding neighbors
    airbnb.at[counter, "commercial_pts_5min_walk"] = nearby_int_pts

#### 10 Minute Walk Case

In [17]:
#create one column for airbnb table
#1. total number of commercial points within 10 minute walk
airbnb["commercial_pts_10min_walk"] = 0

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.007636)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.007636)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #augment counter below if it's an interest point
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new column with total count of corresponding neighbors
    airbnb.at[counter, "commercial_pts_10min_walk"] = nearby_int_pts 

In [18]:
#let's observe the changes
airbnb.head(7)

,id,name,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,review_count,last_review,reviews_per_month,host_listings_count,availability_365,interests_1min_walk,interests_5min_walk,interests_10min_walk,commercial_pts_1min_walk,commercial_pts_5min_walk,commercial_pts_10min_walk
0,31447607.0,SUN FILLED APARTMENT WITH STUNNING RIVER VIEWS,Manhattan,Hell's Kitchen,40.76666,-73.98358,Entire home/apt,225.0,30.0,0.0,NaT,0.00,12.0,343.0,1,38,213,1,12,57
1,31447999.0,Stunning Modern 1br with Private Balcony & Views,Manhattan,Midtown,40.76616,-73.98228,Entire home/apt,225.0,30.0,0.0,NaT,0.00,12.0,365.0,0,40,186,0,19,62
2,31448783.0,Sleek 1 Bedroom Brownstone Apartment in Brooklyn,Brooklyn,Bedford-Stuyvesant,40.67890,-73.91014,Entire home/apt,80.0,2.0,8.0,2019-07-07,2.12,1.0,0.0,1,18,75,0,0,0
3,31450131.0,Apt2- large bedroom.,Brooklyn,Brownsville,40.66262,-73.91575,Private room,59.0,1.0,2.0,2019-05-31,0.42,2.0,53.0,0,19,100,0,0,0
4,31453103.0,Newly Renovated Cozy Private 1 Bedroom Apartment,Brooklyn,East Flatbush,40.64656,-73.95016,Entire home/apt,65.0,2.0,20.0,2019-06-29,3.57,2.0,293.0,0,9,40,0,0,2
5,31457187.0,Brooklyn Bliss V,Brooklyn,East Flatbush,40.65808,-73.93903,Private room,39.0,2.0,3.0,2019-05-08,0.73,2.0,67.0,0,20,61,0,0,0
6,31458142.0,Huge studio in the heart of Kips Bay,Manhattan,Kips Bay,40.74343,-73.98147,Entire home/apt,140.0,1.0,3.0,2019-04-01,0.63,1.0,0.0,3,42,172,1,10,36


### Transportation Interest Points Only
#### 1 Minute Walk Case

In [19]:
#create one column for airbnb table
#total number of transportation interest points within 1 minute walk
airbnb["transport_pts_1min_walk"] = 0

#list of all transportation interest points coordinates
#previous airbnb_coordinates remains unchanged
interest_coordinates = list(zip(interests_transport.latitude, interests_transport.longitude))

#need to join the airbnb coordinates with the interest points coordinates
#to obtain a single list of combined coordinates that can be placed in the k-d tree
#for faster querying
combined_coordinates = interest_coordinates + airbnb_coordinates
tree = cKDTree(combined_coordinates)

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.000757)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.000757)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #and augment either of the two counters by each one found
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new columns with total count of corresponding neighbors
    airbnb.at[counter, "transport_pts_1min_walk"] = nearby_int_pts

#### 5 Minute Walk Case

In [20]:
#create one column for airbnb table
#1. total number of transportation interest points within 5 minute walk
airbnb["transport_pts_5min_walk"] = 0

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.003818)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.003818)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #augment counter below only if it's an interest point
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new column with total count of corresponding neighbors
    airbnb.at[counter, "transport_pts_5min_walk"] = nearby_int_pts

#### 10 Minute Walk Case

In [21]:
#create one column for airbnb table
#1. total number of transportation interest points within 10 minute walk
airbnb["transport_pts_10min_walk"] = 0

#for each airbnb location, let's find the total count of nearby interest points
#and place it into its corresponding column
for counter, point in enumerate(airbnb_coordinates):
    
    #find the proximity of points to our designated point (within radius = 0.007636)
    proximity, proximity_indices = tree.query(point, len(combined_coordinates), p=2, distance_upper_bound=0.007636)
    neighbor_coordinates = []
    
    #get only the points that are within our radius
    for distance_index, distance in zip(proximity_indices, proximity):
        if distance == inf:
            break
        #place the interest points in the neighbor list
        neighbor_coordinates.append(combined_coordinates[distance_index])
    
    #remove first value from list (it's the airbnb point itself)
    neighbor_coordinates.pop(0)
     
    #check whether our neighbor points are airbnbs or interest points
    #augment counter below if it's an interest point
    nearby_int_pts = 0
    
    for i, j in neighbor_coordinates:
        if (i in airbnb_latitude_set) and (j in airbnb_longitude_set):
            continue
        else:
            nearby_int_pts += 1
    
    #update our new column with total count of corresponding neighbors
    airbnb.at[counter, "transport_pts_10min_walk"] = nearby_int_pts 

In [22]:
#let's observe the changes
airbnb.head(7)

,id,name,borough,neighbourhood,latitude,longitude,room_type,price,minimum_nights,review_count,...,availability_365,interests_1min_walk,interests_5min_walk,interests_10min_walk,commercial_pts_1min_walk,commercial_pts_5min_walk,commercial_pts_10min_walk,transport_pts_1min_walk,transport_pts_5min_walk,transport_pts_10min_walk
0,31447607.0,SUN FILLED APARTMENT WITH STUNNING RIVER VIEWS,Manhattan,Hell's Kitchen,40.76666,-73.98358,Entire home/apt,225.0,30.0,0.0,...,343.0,1,38,213,1,12,57,0,0,6
1,31447999.0,Stunning Modern 1br with Private Balcony & Views,Manhattan,Midtown,40.76616,-73.98228,Entire home/apt,225.0,30.0,0.0,...,365.0,0,40,186,0,19,62,0,0,7
2,31448783.0,Sleek 1 Bedroom Brownstone Apartment in Brooklyn,Brooklyn,Bedford-Stuyvesant,40.67890,-73.91014,Entire home/apt,80.0,2.0,8.0,...,0.0,1,18,75,0,0,0,0,0,1
3,31450131.0,Apt2- large bedroom.,Brooklyn,Brownsville,40.66262,-73.91575,Private room,59.0,1.0,2.0,...,53.0,0,19,100,0,0,0,0,0,0
4,31453103.0,Newly Renovated Cozy Private 1 Bedroom Apartment,Brooklyn,East Flatbush,40.64656,-73.95016,Entire home/apt,65.0,2.0,20.0,...,293.0,0,9,40,0,0,2,0,0,0
5,31457187.0,Brooklyn Bliss V,Brooklyn,East Flatbush,40.65808,-73.93903,Private room,39.0,2.0,3.0,...,67.0,0,20,61,0,0,0,0,0,0
6,31458142.0,Huge studio in the heart of Kips Bay,Manhattan,Kips Bay,40.74343,-73.98147,Entire home/apt,140.0,1.0,3.0,...,0.0,3,42,172,1,10,36,0,1,2


We've now updated our Airbnb table with the amount of neighboring interest points within each corresponding radius. It's time to upload all relevant tables to the SQL database.

In [23]:
#push finalized tables back to SQL database
airbnb.to_sql('airbnb', con=database.engine, if_exists='replace', schema='dbo', index=False)
airbnb_stats.to_sql('airbnb_stats', con=database.engine, if_exists='replace', schema='dbo', index=False)
interests.to_sql('interest_points', con=database.engine, if_exists='replace', schema='dbo', index=False)

**Please [click here](https://public.tableau.com/en-us/s/) to view my visualizations in Tableau with the data above.**